[사용법]

1, 2, 3, 4 순서대로 실행하면됨
path는 그때그때 변경

작성자: 김강렬  
수정: 강성우

In [1]:
# 1. [라벨링 폴더 경로 설정 - path 변수에 넣으면 됨]
import json
import sys
import os
 
# 작업할 json파일(라벨링)이 존재하는 폴더 경로 설정
path_dir = 'C:/Users/user/fish_video_disease_bbox_dataset/json/'

# 여기에 원하는 경로를 넣어주면됨
path = path_dir

file_list = os.listdir(path)

print(len(file_list))
print(file_list)
print(type(file_list))

readme_txt = "[데이터 형식]\n"\
+ "굴 객체의 바운딩 박스 좌표(좌측상단xy, 우측하단xy)\n"\
+ "부착생물의 클래스, 바운딩 박스 좌표(좌측상단xy, width, height)\n"\
+ "이미지 크롭 후 부착생물의 클래스, 바운딩박스 좌표(좌측상단xy, width, height)\n"\
+ "temp"
# print(readme_txt)

5
['dtset1_disease.json', 'dtset2_disease.json', 'dtset3_disease.json', 'dtset4_disease.json', 'dtsetV_disease.json']
<class 'list'>


In [ ]:
# 2. 물고기 bbox 데이터 추출: crop_area_data 생성
def find_image_info(images, image_id, info_type):
    info = ''
    for i in images:
        if i['id'] == image_id:
            if info_type == 'file_name':
                info = i['file_name']
            elif info_type == 'width':
                info = i['width']
            elif info_type == 'height':
                info = i['height']
    return info

with open('C:/Users/user/fish_video_disease_bbox_dataset/dtsetV_disease.json', 'r', encoding='utf-8') as f:
        data = json.load(f)  # load JSON
    
images = data['images'] # 이미지 정보 딕셔너리
ants = data['annotations'] # 어노테이션 정보 딕셔너리
save_dir = 'C:/Users/user/fish_video_disease_bbox_dataset/crop_area_data/'

for i in ants:
    if i['diseases_exist'] is True: # 질병이 있는 물고기
        if i['category_id'] == 5:  # 돌돔
            file_name = find_image_info(images, i['image_id'], 'file_name')
            bbox = i['bbox']
            with open(save_dir + file_name[2:-4] + '.txt', 'w') as f:
                f.write(str(bbox[0]) + " " + str(bbox[1]) + " " + str(bbox[0] + bbox[2]) + " " + str(bbox[1] + bbox[3]))
             

In [2]:
# 성우꺼 [YOLOv5 라벨 데이터 생성 - Crop된 굴 이미지 기준]
# Crop된 굴 이미지를 기준으로,
# YOLOv5에서 사용하는 txt 라벨 데이터 만들기

import json

class_index = 0
img_width = 2704 
img_height = 1520
padding = 0 # 이미지 여백
result_bbox = []
save_dir = "C:/Users/user/fish_video_disease_bbox_dataset/crop_label/"
crop_data_path = "C:/Users/user/fish_video_disease_bbox_dataset/crop_area_data/"

# 이미지 번호로 이미지 파일 정보를 찾아서 반환
# images: 이미지 정보 딕셔너리
# image_id: 검색할 이미지 번호
# info_type: 찾을 정보 종류
def find_image_info(images, image_id, info_type):
    info = ''
    for i in images:
        if i['id'] == image_id:
            if info_type == 'file_name':
                info = i['file_name']
            elif info_type == 'width':
                info = i['width']
            elif info_type == 'height':
                info = i['height']
    return info

for file in file_list:
    # 값 추출을 위한 사전작업(파일 오픈 및 접근)
    with open(path + file, "r", encoding="UTF-8") as f:
        json_data = json.load(f)
        
    # annotations 정보 가져오기
    annotations = json_data['annotations']
    img_file_name = ''
    for index in annotations:
        # 질병이 있는 돌돔 
        if index['category_id'] != 5:
            continue
        elif index['diseases_exist'] is True:
            class_index = 0
            img_file_name = find_image_info(json_data['images'], index['image_id'], 'file_name')
        else:
            class_index = 1
            
        # 질병 바운딩박스 추출
        bbox_data = index['diseases_bbox']
#         break

        # crop 데이터(좌측 상단xy, 우측 하단xy 좌표값들) 가져오기.
        with open(crop_data_path + img_file_name[2:-4] + ".txt", "r", encoding="UTF-8") as f:
            crop_data = f.readline().split(" ")
            
        for i in range(0, 4):
            crop_data[i] = crop_data[i].split(".")[0]
            
        try:
            crop_location_x_left = int(crop_data[0]) #int(content[0]) 
            crop_location_y_left = int(crop_data[1]) #int(content[1]) 

            img_width = int(crop_data[2]) - int(crop_data[0]) + (padding * 2)
            img_height = int(crop_data[3]) - int(crop_data[1]) + (padding * 2)

        except:
            print("crop 데이터 없는 파일: ", file)
            continue

        
        # YOLOv5 형식에 맞게 데이터 구성
        x_center = bbox_data[0] + padding - crop_location_x_left + bbox_data[2] / 2
        y_center = bbox_data[1] + padding - crop_location_y_left + bbox_data[3] / 2 # y_center값 구하기

        normalized_x_center = x_center / img_width
        normalized_y_center = y_center / img_height
        normalized_width = bbox_data[2] / img_width
        normalized_height = bbox_data[3] / img_height

        # YOLOv5 라벨형식으로 정규화가 제대로 안된경우,
        # 오류 파일 이름 저장 및 넘어가기
        if normalized_x_center <= 0 or normalized_y_center <= 0:
            with open(save_dir + "normalized_error.txt", 'a') as ne_f:
                ne_f.write(file + "\n")
            print("정규화에러: ", file)
            continue
        if normalized_x_center >= 1 or normalized_y_center >= 1:
            with open(save_dir + "normalized_error.txt", 'a') as ne_f:
                ne_f.write(file + "\n")
            print("정규화에러: ", file)
            continue

        result_bbox.append(normalized_x_center)
        result_bbox.append(normalized_y_center)
        result_bbox.append(normalized_width)
        result_bbox.append(normalized_height)


        # YOLOv5 라벨 형식으로 저장
        with open(save_dir + img_file_name[2:-4] + "_crop.txt", 'w') as s_f: #동일 파일 이름으로 txt 생성
            s_f.write(str(class_index))
            for re_bb in result_bbox:
                s_f.write(" " + str(re_bb))
            s_f.write("\n")
    #         print(result_bbox)
        result_bbox = []
    


In [3]:
# 성우꺼2 [실제 이미지 크롭]
import sys
import os
from PIL import Image
root_path = "C:/Users/user/fish_disease_dataset/multy_class_with_fish_bbox/set/"

target_img_path = root_path + "aug/images/"
target_img_list = os.listdir(target_img_path)

save_img_path = root_path + "crop/images/"

target_area_path = root_path + "aug/labels/"


for file in target_img_list: 
    # 이미지 불러오기
    img = Image.open(target_img_path + file)
    # crop 영역 불러오기
    with open(target_area_path + file[:-4] + ".txt", 'r') as f:
        f.readlines()
#         content = f.readline().split(" ")
#         for i in range(0, 4):
#             content[i] = content[i].split(".")[0]
            
            
#     # 원본 이미지 크롭 후 저장
#     if int(content[0])-padding < 0 or int(content[1])-padding < 0:
#         with open(save_img_path + "img_crop_error.txt", 'a') as ice_f:
#             ice_f.write(file + "\n")
#         print("음수 에러: ", file)
#         continue

#     area = (int(content[0])-padding, int(content[1])-padding, int(content[2])+padding, int(content[3])+padding)

#     cropped_img = img.crop(area) # crop(좌측상단x, y, 우측하단x, y)
#     cropped_img.save(save_img_path + file[:-4] + "_crop.jpg")

C:/Users/user/fish_disease_dataset/multy_class_with_fish_bbox/set/aug/images/216216_objt_rb_2021-01-05_11-31-23-66_003.jpg
C:/Users/user/fish_disease_dataset/multy_class_with_fish_bbox/set/aug/images/216216_objt_rb_2021-01-05_11-31-23-66_003_bright_down.jpg
C:/Users/user/fish_disease_dataset/multy_class_with_fish_bbox/set/aug/images/216216_objt_rb_2021-01-05_11-31-23-66_003_bright_down_out_0.2.jpg
C:/Users/user/fish_disease_dataset/multy_class_with_fish_bbox/set/aug/images/216216_objt_rb_2021-01-05_11-31-23-66_003_out_0.2.jpg
C:/Users/user/fish_disease_dataset/multy_class_with_fish_bbox/set/aug/images/216216_objt_rb_2021-01-05_11-31-23-66_003_rot_15-30.jpg
C:/Users/user/fish_disease_dataset/multy_class_with_fish_bbox/set/aug/images/216216_objt_rb_2021-01-05_11-31-23-66_003_rot_15-30_out_0.2.jpg
C:/Users/user/fish_disease_dataset/multy_class_with_fish_bbox/set/aug/images/216216_objt_rb_2021-01-05_11-31-23-66_003_rot_hf.jpg
C:/Users/user/fish_disease_dataset/multy_class_with_fish_bbox/se